# Tutorial 10 - Merge fixation and blink ids into gaze dataframe

The goal of this tutorial is to teach you how to merge fixation, blink, and gaze data
into one dataframe. The result will allow you to quickly find or filter gaze samples
that belong to fixations and/or blinks.

We will follow these steps:
1. Load the data using `Pandas`
1. For each fixation and blink, find and annotate the corresponding gaze data
    1. Select the start and end timestamp of the corresponding detection
    1. Find all gaze samples whose timestamps fall into that period
    1. Mark these gaze samples accordingly
1. Visualize the resulting dataframe

Please check the documentation of [Pupil Player](https://docs.pupil-labs.com/core/software/pupil-player/#pupil-player)
on how to detect blinks and fixations and how to export data.

This notebook is accompanied by a short example export. You can find it in the
`data/10_merge_fixation_and_blink_ids_into_gaze_dataframe/` subfolder which contains the
three files that are important for this tutorial:

- `gaze_positions.csv`
- `blinks.csv`
- `fixations.csv`

## 1 - Loading Exported Data
Let's start by importing the necessary Python modules and loading the exported data.

In [6]:
import pandas as pd


In [7]:
export_folder = "data/10_merge_fixation_and_blink_ids_into_gaze_dataframe/"
gaze = pd.read_csv(export_folder + "gaze_positions.csv")
blinks = pd.read_csv(export_folder + "blinks.csv")
fixations = pd.read_csv(export_folder + "fixations.csv")


### Gaze data

You can find information on how the exported gaze data looks like in our previous tutorials.
Below, we only preview the data that is necessary for the blink and fixation matching:
The `gaze_timestamp` field.

In [8]:
gaze["gaze_timestamp"].head(3)  # .head(n): only display the first n entries


0    329367.897894
1    329367.905404
2    329367.907907
Name: gaze_timestamp, dtype: float64

### Blink data

The exported section contains three blinks of which we will need the `id`,
`start_timestamp`, and `end_timestamp`.

**Note**: The blink ids do not start at 0 or 1 because the exported data only represents
a short slice of a longer recording.

In [9]:
blinks[["id", "start_timestamp", "end_timestamp"]]  # only display the desired columns


,id,start_timestamp,end_timestamp
0,5,329368.551824,329368.927313
1,6,329372.045221,329372.495808
2,7,329373.668492,329374.104060


### Fixation data

In contrast to the blinks, the exported fixations do not have an `end_timestamp` field.
We can calculate the corresponding values by summing the `start_timestamp` and `duration`
values.

**Note**: `duration` is in milliseconds.

In [10]:
fixations[["id", "start_timestamp", "duration"]].head(3)


,id,start_timestamp,duration
0,65,329367.872861,202.7645
1,66,329368.078129,172.7250
2,67,329368.308429,172.7250


In [11]:
fixations["end_timestamp"] = fixations.start_timestamp + fixations.duration / 1_000

fixations[["id", "start_timestamp", "end_timestamp"]].head(3)  # display result


,id,start_timestamp,end_timestamp
0,65,329367.872861,329368.075625
1,66,329368.078129,329368.250853
2,67,329368.308429,329368.481154


## 2 - Find gaze data for each fixation and blink

We will save the matching blink and fixation ids for each gaze sample in a dedicated
column. To start, we set this column to `None` to indicate that the samples are neither
a blink nor a fixation.

In [12]:
gaze["blink_id"] = None
gaze["fixation_id"] = None


Next, we iterate over all blinks and fixations, extract their start and end timestamps,
and find the corresponding gaze samples.

For this, we use [`pandas.Series.between()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html).

In [13]:
for blink in blinks.itertuples():
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html
    # Return boolean Series equivalent to left <= series <= right.
    # `found_sample_mask` has the size as `gaze` and is True for all gaze samples
    # whose `gaze_timestamp` lies between the blink start end end time.
    found_sample_mask = gaze.gaze_timestamp.between(
        blink.start_timestamp, blink.end_timestamp
    )
    gaze.loc[found_sample_mask, "blink_id"] = blink.id

    # To count the found samples, we sum the boolean values (True = 1, False = 0).
    num_samples = found_sample_mask.sum()
    print(f"Found {num_samples} gaze samples for blink {blink.id}")


Found 124 gaze samples for blink 5
Found 121 gaze samples for blink 6
Found 117 gaze samples for blink 7


In [14]:
for fixation in fixations.itertuples():
    # Here we repeat the procedure
    found_sample_mask = gaze.gaze_timestamp.between(
        fixation.start_timestamp, fixation.end_timestamp
    )
    gaze.loc[found_sample_mask, "fixation_id"] = fixation.id


## 3 - Visualize results

To keep the example clear, we only display a subset of the available gaze dataframe
columns, including the new `blink_id` and `fixation_id` columns.

In [16]:
gaze[
    [
        "gaze_timestamp",
        "confidence",
        "norm_pos_x",
        "norm_pos_y",
        "blink_id",
        "fixation_id",
    ]
]


,gaze_timestamp,confidence,norm_pos_x,norm_pos_y,blink_id,fixation_id
0,329367.897894,0.980058,0.501013,0.489436,None,65
1,329367.905404,0.996493,0.500626,0.489527,None,65
2,329367.907907,0.996493,0.500597,0.490593,None,65
3,329367.910410,0.997756,0.501219,0.491356,None,65
4,329367.915417,0.997756,0.501431,0.490214,None,65
...,...,...,...,...,...,...
2049,329374.814987,0.281357,0.399956,0.300403,None,100
2050,329374.816911,0.752056,0.436452,0.308474,None,100
2051,329374.819993,0.372858,0.411641,0.329027,None,100
2052,329374.825000,0.424030,0.402146,0.330951,None,100
